In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
! pip install seaborn --upgrade

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
df = pd.read_csv("../input/avocado-prices/avocado.csv")

In [ ]:
df

## Two ways to detect outlier

### 1. Use standard deviation
### 2. Use quantile

In [ ]:
sns.set_style("white")

plt.figure(figsize=(12,12))
sns.distplot(df.AveragePrice)
plt.title("Distribution of Average Price",fontsize=14)

### Looks like right skewed

In [ ]:
sm.qqplot(df.AveragePrice,line="s")

## First way : Use property of mean and standard deviation

### 1 std from mean : 68% of data included
### 2 std from mean : 95% of data included
### 3 std from mean : 99.7% of data included

In [ ]:
mean = df.AveragePrice.mean()
std = df.AveragePrice.std()
lower, upper = mean-std*2,mean+std*2 # Use 2*std and it will exclude data that is not included in 95% of data
print("Lower bound : {} Upper bound : {}".format(lower,upper))

In [ ]:
outliers = [x for x in df.AveragePrice if x < lower or x > upper]
outliers

### There is some data that is not included within 95% of data

In [ ]:
df_exclude = df[(df.AveragePrice < upper) | (df.AveragePrice > lower)]

In [ ]:
df_exclude

## Second way : Use quantile and IQR

### Q1 : Data that is located in 25% of total data
### Q2 : Median value of data
### Q3 : Data that is located in 75% of total data

### IQR = Q3 - Q1

### Outlier

### Upper bound : 1.5*IQR + Q3
### Lower bound : 1.5*IQR - Q1




In [ ]:
quantile = np.quantile(df.AveragePrice,[0.25,0.5,0.75,1]) # Use numpy quantile
IQR = quantile[2] - quantile[0] # Calculate IQR through third quantile - first quantile
upper = 1.5*IQR + quantile[2]
lower = quantile[0] - 1.5*IQR

print("Upper bound : {} Lower bound : {}".format(upper,lower))

outlier = [x for x in df.AveragePrice if x < lower or x>upper]

In [ ]:
outlier

In [ ]:
df_exclude2 = df[(df.AveragePrice > lower) | (df.AveragePrice < upper)]
df_exclude2

## Data Normalization

### It is important to check whether data follow normoal distribution before we do modeling
### There is one easy way to do normalization, use logarithmic

In [ ]:
sns.set_style("white")

plt.figure(figsize=(12,12))
sns.distplot(df.AveragePrice)
plt.title("Distribution of Average Price",fontsize=14)

## We checked before that this price data distributed right skewed

In [ ]:
log_data = np.log(df.AveragePrice+1)

sns.set_style("white")

plt.figure(figsize=(12,12))
sns.distplot(log_data)
plt.title("Distribution of Average Price",fontsize=14)

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,7))
sm.qqplot(df.AveragePrice,line="s",ax=ax[0])
ax[0].set_title("Before logarithmic")
sm.qqplot(log_data,line="s",ax=ax[1])
ax[1].set_title("After logarithmic")

### It looks much closer to normal distribution after doing logarithmic